# Module 17 Checkpoint 3 (Data cleaning 2: missing values) Assignment

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

Start by loading the remote database into a dataframe

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

education_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

# Determine all the variable types and find the fraction of the missing values for each variable

In [3]:
# Determine variable types:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

In [4]:
# Find the fraction of missing values:
education_df.isnull().sum()/education_df.isnull().count()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
OTHER_EXPENDITURE               0.176273
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
GRADES_ALL_G                    0.115952
AVG_MATH_4_SCORE                0.640751
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_READING_8_SC

# Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

The most useful variables that can be filled in are the those that are numeric (floats) and *not* already aggregated (i.e. the final 4 variables appear to be averages). Depending on how the data is organized (check if it appears to be already sorted by year and/or state), records with blank year and/or state values may need to be discarded, since they would be difficult to accurately impute if the data is arranged randomly.

In [5]:
# Duplicate the dataframe in case I do something stupid, so the original remains intact.
educationdup_df = education_df.copy()

# Determine which columns to apply imputation to (the non-aggregated floats)
imputable_columns = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", 
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE", 
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G", 
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G", 
             "GRADES_9_12_G", "GRADES_ALL_G"]

# For each column selected, replace all null values with the mean (aggregated over the entire column)
for column in imputable_columns:
    educationdup_df.loc[:, column].fillna(educationdup_df.loc[:, column].mean(), inplace=True)

# Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

In [6]:
# Make a new duplicate to store this attempt
educationdup2_df = education_df.copy()

# Find all unique year values to iterate over
years = educationdup2_df["YEAR"].unique()

# In each column, for each year, fill nulls with the mean of that specific year
for column in imputable_columns:
    for year in years:
        educationdup2_df.loc[educationdup2_df["YEAR"] == year, column].fillna(
            educationdup2_df[educationdup2_df["YEAR"] == year][column].mean(), inplace=True)

# This time, fill in the missing values using interpolation (extrapolation).

In [7]:
# Another storage duplicate
educationdup3_df = education_df.copy()

# Use interpolation (with default settings because I don't know how to really use this yet!)
for column in imputable_columns:
    educationdup3_df.loc[:, column].interpolate(inplace=True)

# Compare your results for the second, third and the fourth questions. Do you find any meaningful differences?

Compare basic statistics for each of the 3 dataframes that has been modified from the original.

In [8]:
# Use .describe to compare each of the columns where imputation was used!
for column in imputable_columns:
    print("Statistics for columns: {}".format(column))
    print(pd.concat([educationdup_df[column], educationdup2_df[column], educationdup3_df[column]], axis=1).describe())

Statistics for columns: ENROLL
             ENROLL        ENROLL        ENROLL
count  1.492000e+03  1.229000e+03  1.456000e+03
mean   9.159308e+05  9.159308e+05  1.005984e+06
std    9.667725e+05  1.065280e+06  1.009158e+06
min    4.386600e+04  4.386600e+04  4.386600e+04
25%    3.150940e+05  2.583140e+05  2.973995e+05
50%    8.204140e+05  6.483130e+05  7.540845e+05
75%    9.211780e+05  1.014528e+06  1.595024e+06
max    6.307022e+06  6.307022e+06  6.307022e+06
Statistics for columns: TOTAL_REVENUE
       TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE
count   1.492000e+03   1.280000e+03   1.492000e+03
mean    9.092082e+06   9.092082e+06   1.111164e+07
std     1.087818e+07   1.174519e+07   1.200155e+07
min     4.656500e+05   4.656500e+05   4.656500e+05
25%     2.546261e+06   2.186305e+06   2.546261e+06
50%     6.359310e+06   5.079546e+06   6.351760e+06
75%     9.276958e+06   1.085985e+07   1.803650e+07
max     8.921726e+07   8.921726e+07   8.921726e+07
Statistics for columns: FEDERAL_REVENUE


       GRADES_9_12_G  GRADES_9_12_G  GRADES_9_12_G
count   1.492000e+03   1.361000e+03   1.492000e+03
mean    2.470714e+05   2.470714e+05   2.286678e+05
std     2.938773e+05   3.077056e+05   3.005815e+05
min     1.808000e+03   1.808000e+03   1.808000e+03
25%     5.919750e+04   5.147100e+04   3.950675e+04
50%     1.817010e+05   1.642600e+05   1.472355e+05
75%     2.825390e+05   2.905020e+05   2.843505e+05
max     2.013687e+06   2.013687e+06   2.013687e+06
Statistics for columns: GRADES_ALL_G
       GRADES_ALL_G  GRADES_ALL_G  GRADES_ALL_G
count  1.492000e+03  1.319000e+03  1.492000e+03
mean   8.024415e+05  8.024415e+05  7.478782e+05
std    9.126522e+05  9.707031e+05  9.291585e+05
min    7.254000e+03  7.254000e+03  7.254000e+03
25%    2.293578e+05  1.810650e+05  1.718665e+05
50%    6.473260e+05  5.503420e+05  4.591248e+05
75%    8.724220e+05  9.282755e+05  8.860948e+05
max    5.944746e+06  5.944746e+06  5.944746e+06
